# Exercise 3: Durable Agent — Solution

Complete implementation combining **OpenAI Agents SDK** with **Temporal** for durability.

This solution demonstrates how Temporal automatically retries failed operations, making your AI agents resilient to network failures and other transient errors.

## Architecture

This solution follows a **4-component pattern** that mirrors production Temporal applications:

1. **`activities.py`** - Defines the weather activity
2. **`workflow.py`** - Defines the workflow orchestration
3. **`worker.py`** - Runs the worker that executes workflows/activities
4. **`starter.py`** - Executes the workflow

Each component is implemented in a separate Jupyter cell below for clarity.

### Flow Diagram 

```
User Query 
    ↓
Temporal Workflow (orchestration layer) 
    ↓
Activity: LLM interprets query and may choose tool 
    ↓
[Tool chosen?]
    ↓
Activity: Execute get_weather tool 
    ↓
External API Call (NWS) 
    ↓
Data returned to Agent 
    ↓
Agent leverages LLM to generate response 
    ↓
Return to user [OK]
```

**Each activity can retry independently, and the entire flow is durable!** 


## Setup

Before running this solution, ensure you have:

1. **Temporal Server Running:**
   - Open `temporal_installation.ipynb` in the project root
   - Run all cells to install Temporal CLI and start the dev server
   - **Verify:** In Codespaces, go to the **Ports** tab → Find port **8233** → Click the **Globe icon** 

2. **Dependencies Installed:**
   - Required packages should already be installed from the main setup

3. **API Key Configured:**
   - Ensure your `.env` file in the project root contains your OpenAI API key


In [1]:
%pip install --quiet temporalio openai-agents httpx nest-asyncio pytz

# Import all required modules
import asyncio
from datetime import datetime, timedelta

import httpx
import nest_asyncio
import pytz
from agents import Agent, Runner
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.contrib import openai_agents
from temporalio.contrib.openai_agents import ModelActivityParameters, OpenAIAgentsPlugin
from temporalio.worker import Worker

print("[OK] All imports successful")


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ All imports successful


## Component 1: `activities.py`

Define the weather activity that fetches alerts from the National Weather Service API.

In [2]:
async def make_api_call(state: str) -> dict:
    """Make API call to fetch weather alerts from National Weather Service."""
    # Set User-Agent header required by National Weather Service API
    headers = {"User-Agent": "Temporal-Agents-Workshop/1.0 (educational@example.com)"}
    # Create async HTTP client with 10-second timeout to prevent hanging
    async with httpx.AsyncClient(timeout=10) as client:
        # Make GET request to NWS alerts endpoint for the specified state
        r = await client.get(
            f"https://api.weather.gov/alerts/active/area/{state}",
            headers=headers
        )
        # Raise exception if request fails (4xx or 5xx) - Temporal will auto-retry
        r.raise_for_status()
        # Parse JSON response into Python dictionary
        data = r.json()

    # Return raw data for processing
    return data

async def make_api_call_bug(state: str) -> dict:
    """Make API call with intentional bug - simulates a failing service."""
    # Simulate a service failure by raising an exception
    raise Exception("Simulated API failure: Service temporarily unavailable")

@activity.defn(name="get_weather")
async def get_weather(state: str) -> dict:
    """Fetch active NWS alerts for a 2-letter US state code (e.g., 'CA')."""
    # Call the API function to get weather data
    data = await make_api_call(state)

    # Broken API call to simulate failure
    #data = await make_api_call_bug(state)

    # Initialize empty list to collect alert information
    alerts = []
    # Loop through first 5 features (weather alerts) in the response
    for f in (data.get("features") or [])[:5]:
        # Extract properties object from each feature (contains alert details)
        p = f.get("properties", {})
        # Build structured alert dictionary with key information
        alerts.append({
            "event": p.get("event"),  # Alert type (e.g., "Flash Flood Warning")
            "headline": p.get("headline"),  # Human-readable alert headline
            "severity": p.get("severity"),  # Severity level (e.g., "Severe", "Moderate")
            "area": p.get("areaDesc"),  # Geographic area affected by alert
        })

    # Return structured response with state, count, and alerts for LLM to interpret
    return {"state": state.upper(), "count": len(alerts), "alerts": alerts}

print("[OK] Activity 'get_weather' defined")

✅ Activity 'get_weather' defined


## Component 2: `workflow.py`

Define the workflow that orchestrates the agent execution.

**Key points:**
- Workflow creates the Agent with tools and instructions
- Converts Temporal activity to Agent tool using `activity_as_tool()`
- Runs the agent with the user query
- Returns the final agent response

In [3]:
# Define which task queue this workflow will use for communication
TASK_QUEUE = "agents-sdk-queue"

@workflow.defn(sandboxed=False)  # Disable sandbox for Jupyter compatibility
class WeatherAgentWorkflow:  # Define workflow class for orchestrating the agent
    @workflow.run  # Mark this method as the workflow entry point
    async def run(self, user_query: str) -> str:  # Accept user query, return agent response
        # Create the Agent inside the workflow with tools and instructions
        agent = Agent(
            name="Weather Assistant",  # Name shown in OpenAI traces
            # Define agent's role and behavior when handling user queries
            instructions=(
                "You are a helpful assistant that explains current weather alerts for U.S. states. "
                "When the user asks about alerts, call the get_weather tool, then summarize results."
            ),
            # Provide list of tools the agent can use
            tools=[
                # Convert Temporal activity to Agent tool for durable execution
                openai_agents.workflow.activity_as_tool(
                    get_weather,  # The activity function to wrap
                    start_to_close_timeout=timedelta(seconds=10),  # Max time for activity execution
                )
            ],
        )

        # Run the agent with user query - tool calls execute as Temporal activities
        result = await Runner().run(agent, user_query)

        # Safely extract final output from result object (handles different result types)
        return getattr(result, "final_output", str(result))

print(f"[OK] Workflow 'WeatherAgentWorkflow' defined (task queue: {TASK_QUEUE})")

✅ Workflow 'WeatherAgentWorkflow' defined (task queue: agents-sdk-queue)


## Component 3: `worker.py`

Run the worker that polls for and executes workflow/activity tasks.

**Key points:**
- Connects to Temporal with `OpenAIAgentsPlugin` (required)
- Configures `ModelActivityParameters` for AI model timeout settings
- Uses `SandboxedWorkflowRunner` with passthrough for `httpx`
- Runs in background as an async task in Jupyter

In [4]:
async def run_worker():  # Define async function to start and run the worker
    """Start a Temporal worker that listens for workflow and activity tasks."""
    # Connect to local Temporal server with OpenAI Agents SDK plugin
    client = await Client.connect(
        "localhost:7233",  # Temporal server address
        # Register plugins to extend Temporal's functionality
        plugins=[
            # Plugin that enables OpenAI Agents SDK integration with Temporal
            OpenAIAgentsPlugin(
                # Configure timeout settings for AI model inference activities
                model_params=ModelActivityParameters(
                    start_to_close_timeout=timedelta(seconds=30)  # Max time for LLM calls
                )
            )
        ],
    )

    # Create worker that polls the task queue for work
    worker = Worker(
        client,  # Use the connected Temporal client
        task_queue=TASK_QUEUE,  # Which queue to poll for tasks
        workflows=[WeatherAgentWorkflow],  # List of workflows this worker can execute
        activities=[get_weather],  # List of activities this worker can execute
    )

    print(f"[OK] Worker started on task queue: {TASK_QUEUE}")
    print("   Listening for workflow and activity tasks...")
    # Start polling and executing tasks (blocks until stopped)
    await worker.run()

# Apply nest_asyncio to allow nested event loops in Jupyter
# Allow async functions to be called from within other async contexts
# This is needed because Jupyter notebooks already run in an event loop
nest_asyncio.apply()
# Create background task to run worker without blocking the notebook
# Create a task that runs the worker concurrently with other notebook operations
worker_task = asyncio.create_task(run_worker())
print(" Worker running in background")

🔄 Worker running in background


✅ Worker started on task queue: agents-sdk-queue
   Listening for workflow and activity tasks...


Error getting response: Error code: 401 - {'error': {'message': 'Missing bearer or basic authentication in header', 'type': 'invalid_request_error', 'param': None, 'code': None}}. (request_id: req_f3305a8b29f81a88205b59220da47069)
Completing activity as failed ({'activity_id': '1', 'activity_type': 'invoke_model_activity', 'attempt': 1, 'namespace': 'default', 'task_queue': 'agents-sdk-queue', 'workflow_id': 'weather-wed-nov-12-035451est', 'workflow_run_id': '019a79d9-8c2a-7dbd-862d-229c8b710f3a', 'workflow_type': 'WeatherAgentWorkflow'})
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.11/site-packages/temporalio/contrib/openai_agents/_invoke_model_activity.py", line 216, in invoke_model_activity
    return await model.get_response(
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vscode/.local/lib/python3.11/site-packages/agents/models/openai_responses.py", line 90, in get_response
    response = await self._fetch_response(
               ^^^^^^^^^^^^^^^^^

## Component 4: `starter.py`

Running a Temporal Workflow.

### Executing the Workflow
* Temporal Workflows are executed indirectly
* Request execution from the Temporal Service
* You do this with the Temporal Client

In [ ]:
async def run_solution():  # Define async function to execute the workflow
    """Execute the weather agent workflow."""
    # Define the user query to send to the agent
    query = "What weather alerts are active in CA?"

    # Generate workflow ID with EST timestamp for human-readable tracking
    est = pytz.timezone('US/Eastern')  # Create EST timezone object
    now = datetime.now(est)  # Get current time in EST
    # Format timestamp as readable string with day-month-date-time pattern
    workflow_id = f"weather-{now.strftime('%a-%b-%d-%I%M%S').lower()}est"

    # Connect to Temporal server with OpenAI Agents SDK plugin
    client = await Client.connect(
        "localhost:7233",  # Temporal server address
        plugins=[OpenAIAgentsPlugin()]  # Enable OpenAI Agents integration
    )

    print(f" Starting workflow: {workflow_id}")
    print(f" Query: {query}\n")

    # Start the workflow (non-blocking) and get handle for tracking
    handle = await client.start_workflow(
        WeatherAgentWorkflow.run,  # Workflow method to execute
        query,  # User query parameter passed to workflow
        id=workflow_id,  # Unique workflow ID for tracking in Temporal UI
        task_queue=TASK_QUEUE  # Queue where worker will pick up this workflow
    )

    print(f"[OK] Workflow started: {handle.id}")
    # Print Temporal UI link for observing workflow execution
    print(f"🔗 View in Temporal UI: http://localhost:8233/namespaces/default/workflows/{workflow_id}\n")
    print(" Waiting for agent response...\n")

    # Wait for workflow to complete and get result (blocking)
    result = await handle.result()

    print("=" * 60)
    print(" Agent Response:")
    print("=" * 60)
    print(result)
    print("=" * 60)

# Run the solution with Jupyter-specific async handling
try:
    # Try to get existing event loop (Jupyter has one running)
    loop = asyncio.get_running_loop()
    # Execute in existing loop
    await run_solution()
except RuntimeError:
    # If no loop exists, create new one and run
    asyncio.run(run_solution())

🚀 Starting workflow: weather-wed-nov-12-035451est
📝 Query: What weather alerts are active in CA?

✅ Workflow started: weather-wed-nov-12-035451est
🔗 View in Temporal UI: http://localhost:8233/namespaces/default/workflows/weather-wed-nov-12-035451est

⏳ Waiting for agent response...



WorkflowFailureError: Workflow execution failed

OPENAI_API_KEY is not set, skipping trace export


## 5. Practicing Durable Execution

* Durable execution is crash-proof execution
* Retries upon failure
* Maintains application state, resuming after a crash at the point of failure
* Can run across a multitude of processes, even on different machines

### Simulating Failure and Recovery

Let's practice experiencing failure and recovery firsthand. 

This will demonstrate:

* How Activities automatically retry on failure
* How Temporal preserves state across Worker restarts
* How you can fix bugs without losing progress

### Step 1: Simulate a bug in your LLM call

In `activities.py`,
comment out:

```py
# data = await make_api_call(state)
```

and uncomment the following line. This will simulate a bad network connection.

```py
data = await make_api_call_bug(state)
```

### Step 2: Execute the Workflow

Follow all steps in Component 1 through 4 to execute all of the code blocks.

### Step 3: Observe automatic retries in web UI

Go to your Temporal UI and note how the `get_weather()` activity is being retried.

### Step 4: Fix the error and run the activity

Now let's "fix" our simulated failure by removing the error. In a real scenario, this could be:

* A database coming back online
* An API endpoint being fixed
* A network issue being resolved

In `activities.py` comment out:

```py
# data = await make_api_call_bug(state)
```

and uncomment the call that actually works. Simulating a "bug fix".

```py
data = await make_api_call(state)
```

In [ ]:
async def make_api_call(state: str) -> dict:
    """Make API call to fetch weather alerts from National Weather Service."""
    # Set User-Agent header required by National Weather Service API
    headers = {"User-Agent": "Temporal-Agents-Workshop/1.0 (educational@example.com)"}
    # Create async HTTP client with 10-second timeout to prevent hanging
    async with httpx.AsyncClient(timeout=10) as client:
        # Make GET request to NWS alerts endpoint for the specified state
        r = await client.get(f"https://api.weather.gov/alerts/active/area/{state}", headers=headers)
        # Raise exception if request fails (4xx or 5xx) - Temporal will auto-retry
        r.raise_for_status()
        # Parse JSON response into Python dictionary
        data = r.json()

    # Return raw data for processing
    return data


async def make_api_call_bug(state: str) -> dict:
    """Make API call with intentional bug - simulates a failing service."""
    # Simulate a service failure by raising an exception
    raise Exception("Simulated API failure: Service temporarily unavailable")


@activity.defn(name="get_weather")
async def get_weather(state: str) -> dict:
    """Fetch active NWS alerts for a 2-letter US state code (e.g., 'CA')."""
    # Call the API function to get weather data
    data = await make_api_call(state)

    # Broken API call to simulate failure
    # data = await make_api_call_bug(state)

    # Initialize empty list to collect alert information
    alerts = []
    # Loop through first 5 features (weather alerts) in the response
    for f in (data.get("features") or [])[:5]:
        # Extract properties object from each feature (contains alert details)
        p = f.get("properties", {})
        # Build structured alert dictionary with key information
        alerts.append(
            {
                "event": p.get("event"),  # Alert type (e.g., "Flash Flood Warning")
                "headline": p.get("headline"),  # Human-readable alert headline
                "severity": p.get("severity"),  # Severity level (e.g., "Severe", "Moderate")
                "area": p.get("areaDesc"),  # Geographic area affected by alert
            }
        )

    # Return structured response with state, count, and alerts for LLM to interpret
    return {"state": state.upper(), "count": len(alerts), "alerts": alerts}


print("[OK] Activity 'get_weather' defined")


## Step 5: Restart the worker with fixed code

In [ ]:
# Kill the old worker and start the new one
import asyncio

x = worker_task.cancel()
worker_task = asyncio.create_task(run_worker())
print("New worker started!")


### Step 6: Observe Successful Completion

***Refresh your Web UI and observe:***

The `get_weather()` Activity now completes successfully!
The entire Workflow shows Completed status


**What just happened?**

* Your Workflow preserved all state through the failure
* The expensive llm_call was never re-executed (saving you money!)
* When you fixed the bug, Temporal automatically continued from where it left off
* No manual intervention needed - just fix the code and restart the Worker

**This is the power of durable execution!** In production, this means:

* API outages don't lose your progress
* You can deploy bug fixes without restarting workflows
* Your users never lose work
* You never pay twice for the same LLM call